In [1]:
import cx_Oracle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
plt.rc('font', family = 'Malgun Gothic') # 한글 입력
plt.rcParams['axes.unicode_minus'] = False # 글꼴 변경시 마이너스 표기 가능으로 처리

In [10]:
# 기존고객

con = cx_Oracle .connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()
sql ='''select a.고객번호, 연도, 분기, sum(구매금액) "총구매금액"
from lcl a
join 분기기준고객 b on a.고객번호 = b.고객번호
group by a.고객번호, 연도, 분기
order by a.고객번호, 연도, 분기'''
cursor.execute(sql)
x = cursor.fetchall()
columns = ['고객번호','연도','분기','총구매금액']
df = pd.DataFrame(x,columns=columns)
cursor.close()
con.commit()
con.close()

display(df)

,고객번호,연도,분기,총구매금액
0,00001,2014,1분기,9141590
1,00001,2014,2분기,10783765
2,00001,2014,3분기,10781550
3,00001,2014,4분기,23334762
4,00001,2015,1분기,9302985
...,...,...,...,...
150923,19337,2014,4분기,765510
150924,19337,2015,1분기,534370
150925,19337,2015,2분기,660260
150926,19337,2015,3분기,1216470


In [11]:
# 비기존고객

con = cx_Oracle .connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()
sql ='''select a.고객번호, 연도, 분기, sum(구매금액) "총구매금액"
from lcl a
join
(select 고객번호 from lcl
minus
select 고객번호 from 분기기준고객) b on a.고객번호 = b.고객번호
group by a.고객번호, 연도, 분기
order by a.고객번호, 연도, 분기'''
cursor.execute(sql)
x = cursor.fetchall()
columns = ['고객번호','연도','분기','총구매금액']
df1 = pd.DataFrame(x,columns=columns)
cursor.close()
con.commit()
con.close()

display(df1)

,고객번호,연도,분기,총구매금액
0,00005,2014,1분기,3145330
1,00005,2014,2분기,2314820
2,00005,2014,3분기,1827290
3,00005,2014,4분기,398440
4,00005,2015,1분기,2124430
...,...,...,...,...
2962,19382,2015,3분기,1266187
2963,19382,2015,4분기,1308847
2964,19383,2015,2분기,27000
2965,19383,2015,3분기,1092360


,고객번호,연도,분기,총구매금액
0,00005,2014,1분기,3145330
1,00005,2014,2분기,2314820
2,00005,2014,3분기,1827290
3,00005,2014,4분기,398440
4,00005,2015,1분기,2124430
...,...,...,...,...
2962,19382,2015,3분기,1266187
2963,19382,2015,4분기,1308847
2964,19383,2015,2분기,27000
2965,19383,2015,3분기,1092360


In [12]:
cusid = list(set(df['고객번호']))
cusid.sort()
df_re = pd.DataFrame()
df_re['고객번호'] = cusid
for i in range(1, 5):
    df_re[f'14_{i}이용금액'] = np.array(df.loc[(df['연도'] == 2014) & (df['분기'] == f'{i}분기'), '총구매금액'])
for i in range(1, 5):
    df_re[f'15_{i}이용금액'] = np.array(df.loc[(df['연도'] == 2015) & (df['분기'] == f'{i}분기'), '총구매금액'])
for i in range(1, 5):
    df_re[f'14_{i}'] = 1
for i in range(1, 5):
    df_re[f'15_{i}'] = 1
df_re

,고객번호,14_1이용금액,14_2이용금액,14_3이용금액,14_4이용금액,15_1이용금액,15_2이용금액,15_3이용금액,15_4이용금액,14_1,14_2,14_3,14_4,15_1,15_2,15_3,15_4
0,00001,9141590,10783765,10781550,23334762,9302985,8595380,7806580,5295938,1,1,1,1,1,1,1,1
1,00002,14123470,10199760,14841180,7080920,18481480,9310130,13842600,12001220,1,1,1,1,1,1,1,1
2,00003,302782,548433,648188,394366,279310,312970,421017,258101,1,1,1,1,1,1,1,1
3,00004,1309643,3732463,2434410,1823100,1737292,1919645,2127112,1610145,1,1,1,1,1,1,1,1
4,00006,2166006,3115284,3163273,2950816,3619234,2769821,3518094,3203889,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18861,19333,613130,913988,578210,455450,850690,553590,252010,520430,1,1,1,1,1,1,1,1
18862,19334,834049,2705917,2575628,2819868,1699930,767309,484920,465910,1,1,1,1,1,1,1,1
18863,19335,937734,2783122,2320105,2042511,2199423,2852695,1553645,2011244,1,1,1,1,1,1,1,1
18864,19336,449600,3321870,3160370,5886390,2896520,3625250,6322150,4948690,1,1,1,1,1,1,1,1


In [13]:
cusid = list(set(df1['고객번호']))
cusid.sort()
df1_re = pd.DataFrame()
df1_re['고객번호'] = cusid

for i in range(1, 5):
    for j in cusid:
        try:
            df1_re.loc[df1_re['고객번호'] == j, f'14_{i}이용금액'] = int(df1.loc[(df1['연도'] == 2014) & (df1['고객번호'] == j) & (df1['분기'] == f'{i}분기'), '총구매금액'])
        except:
            pass
for i in range(1, 5):
    for j in cusid:
        try:
            df1_re.loc[df1_re['고객번호'] == j, f'15_{i}이용금액'] = int(df1.loc[(df1['연도'] == 2015) & (df1['고객번호'] == j) & (df1['분기'] == f'{i}분기'), '총구매금액'])
        except:
            pass
        
df1_re.fillna(0, inplace = True)

for i in range(1, 5):
    df1_re[f'14_{i}'] = df1_re[f'14_{i}이용금액']
for i in range(1, 5):
    df1_re[f'15_{i}'] = df1_re[f'15_{i}이용금액']

display(df1_re.head(20))

,고객번호,14_1이용금액,14_2이용금액,14_3이용금액,14_4이용금액,15_1이용금액,15_2이용금액,15_3이용금액,15_4이용금액,14_1,14_2,14_3,14_4,15_1,15_2,15_3,15_4
0,00005,3145330.0,2314820.0,1827290.0,398440.0,2124430.0,810500.0,0.0,2616260.0,3145330.0,2314820.0,1827290.0,398440.0,2124430.0,810500.0,0.0,2616260.0
1,00142,192000.0,50690.0,0.0,196350.0,52000.0,52000.0,132000.0,192750.0,192000.0,50690.0,0.0,196350.0,52000.0,52000.0,132000.0,192750.0
2,00174,386500.0,0.0,0.0,0.0,0.0,38700.0,0.0,0.0,386500.0,0.0,0.0,0.0,0.0,38700.0,0.0,0.0
3,00244,0.0,1990300.0,2286170.0,2574940.0,4637500.0,8482620.0,11778810.0,7247770.0,0.0,1990300.0,2286170.0,2574940.0,4637500.0,8482620.0,11778810.0,7247770.0
4,00460,2619155.0,7517110.0,3356540.0,9215640.0,4531070.0,2069850.0,0.0,0.0,2619155.0,7517110.0,3356540.0,9215640.0,4531070.0,2069850.0,0.0,0.0
5,00905,6016120.0,7959539.0,5109965.0,6562750.0,5424630.0,6562458.0,195410.0,0.0,6016120.0,7959539.0,5109965.0,6562750.0,5424630.0,6562458.0,195410.0,0.0
6,00985,1109600.0,9509140.0,3352740.0,17555490.0,3137570.0,246880.0,0.0,27420.0,1109600.0,9509140.0,3352740.0,17555490.0,3137570.0,246880.0,0.0,27420.0
7,01067,0.0,3860470.0,4838929.0,7744325.0,6048249.0,4208607.0,6645513.0,8972242.0,0.0,3860470.0,4838929.0,7744325.0,6048249.0,4208607.0,6645513.0,8972242.0
8,01093,0.0,244180.0,0.0,1433628.0,1152153.0,1252889.0,1157264.0,1154732.0,0.0,244180.0,0.0,1433628.0,1152153.0,1252889.0,1157264.0,1154732.0
9,01121,0.0,7893500.0,2803580.0,12643860.0,9104960.0,9728630.0,12799900.0,10263320.0,0.0,7893500.0,2803580.0,12643860.0,9104960.0,9728630.0,12799900.0,10263320.0


In [14]:
df1_re.iloc[:, 1:] = df1_re.iloc[:, 1:].astype(int)
for i in [14, 15]:
    for j in range(1, 5):
        df1_re[f'{i}_{j}'][df1_re[f'{i}_{j}'] > 0] = 1
df1_re

,고객번호,14_1이용금액,14_2이용금액,14_3이용금액,14_4이용금액,15_1이용금액,15_2이용금액,15_3이용금액,15_4이용금액,14_1,14_2,14_3,14_4,15_1,15_2,15_3,15_4
0,00005,3145330,2314820,1827290,398440,2124430,810500,0,2616260,1,1,1,1,1,1,0,1
1,00142,192000,50690,0,196350,52000,52000,132000,192750,1,1,0,1,1,1,1,1
2,00174,386500,0,0,0,0,38700,0,0,1,0,0,0,0,1,0,0
3,00244,0,1990300,2286170,2574940,4637500,8482620,11778810,7247770,0,1,1,1,1,1,1,1
4,00460,2619155,7517110,3356540,9215640,4531070,2069850,0,0,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,19379,0,0,130300,1060320,1131020,1264900,805550,1387190,0,0,1,1,1,1,1,1
513,19380,0,0,0,522700,486830,351120,368907,2183420,0,0,0,1,1,1,1,1
514,19381,0,0,0,0,1124750,1672421,1867501,1375997,0,0,0,0,1,1,1,1
515,19382,0,0,0,0,786399,1040340,1266187,1308847,0,0,0,0,1,1,1,1


In [15]:
display(df_re)
display(df1_re)

,고객번호,14_1이용금액,14_2이용금액,14_3이용금액,14_4이용금액,15_1이용금액,15_2이용금액,15_3이용금액,15_4이용금액,14_1,14_2,14_3,14_4,15_1,15_2,15_3,15_4
0,00001,9141590,10783765,10781550,23334762,9302985,8595380,7806580,5295938,1,1,1,1,1,1,1,1
1,00002,14123470,10199760,14841180,7080920,18481480,9310130,13842600,12001220,1,1,1,1,1,1,1,1
2,00003,302782,548433,648188,394366,279310,312970,421017,258101,1,1,1,1,1,1,1,1
3,00004,1309643,3732463,2434410,1823100,1737292,1919645,2127112,1610145,1,1,1,1,1,1,1,1
4,00006,2166006,3115284,3163273,2950816,3619234,2769821,3518094,3203889,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18861,19333,613130,913988,578210,455450,850690,553590,252010,520430,1,1,1,1,1,1,1,1
18862,19334,834049,2705917,2575628,2819868,1699930,767309,484920,465910,1,1,1,1,1,1,1,1
18863,19335,937734,2783122,2320105,2042511,2199423,2852695,1553645,2011244,1,1,1,1,1,1,1,1
18864,19336,449600,3321870,3160370,5886390,2896520,3625250,6322150,4948690,1,1,1,1,1,1,1,1


,고객번호,14_1이용금액,14_2이용금액,14_3이용금액,14_4이용금액,15_1이용금액,15_2이용금액,15_3이용금액,15_4이용금액,14_1,14_2,14_3,14_4,15_1,15_2,15_3,15_4
0,00005,3145330,2314820,1827290,398440,2124430,810500,0,2616260,1,1,1,1,1,1,0,1
1,00142,192000,50690,0,196350,52000,52000,132000,192750,1,1,0,1,1,1,1,1
2,00174,386500,0,0,0,0,38700,0,0,1,0,0,0,0,1,0,0
3,00244,0,1990300,2286170,2574940,4637500,8482620,11778810,7247770,0,1,1,1,1,1,1,1
4,00460,2619155,7517110,3356540,9215640,4531070,2069850,0,0,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,19379,0,0,130300,1060320,1131020,1264900,805550,1387190,0,0,1,1,1,1,1,1
513,19380,0,0,0,522700,486830,351120,368907,2183420,0,0,0,1,1,1,1,1
514,19381,0,0,0,0,1124750,1672421,1867501,1375997,0,0,0,0,1,1,1,1
515,19382,0,0,0,0,786399,1040340,1266187,1308847,0,0,0,0,1,1,1,1


In [22]:
df_re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18866 entries, 0 to 18865
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   고객번호      18866 non-null  object
 1   14_1이용금액  18866 non-null  int64 
 2   14_2이용금액  18866 non-null  int64 
 3   14_3이용금액  18866 non-null  int64 
 4   14_4이용금액  18866 non-null  int64 
 5   15_1이용금액  18866 non-null  int64 
 6   15_2이용금액  18866 non-null  int64 
 7   15_3이용금액  18866 non-null  int64 
 8   15_4이용금액  18866 non-null  int64 
 9   14_1      18866 non-null  int64 
 10  14_2      18866 non-null  int64 
 11  14_3      18866 non-null  int64 
 12  14_4      18866 non-null  int64 
 13  15_1      18866 non-null  int64 
 14  15_2      18866 non-null  int64 
 15  15_3      18866 non-null  int64 
 16  15_4      18866 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.4+ MB


In [32]:
df_re.to_csv('분기별기존고객이용금액.txt', sep = '\t')
df1_re.to_csv('분기별비기존고객이용금액.csv', encoding = 'cp949')

In [37]:
df2 = pd.read_csv('분기별기존고객이용금액.txt', sep = '\t', index_col = 0)

In [64]:
a = [i.str.zfill(5) for i in df2['고객번호']]
a

AttributeError: 'int' object has no attribute 'str'

In [77]:
df2['고객번호'] = df2['고객번호'].astype(str)
df2['고객번호'].str.zfill(5)

0        00001
1        00002
2        00003
3        00004
4        00006
         ...  
18861    19333
18862    19334
18863    19335
18864    19336
18865    19337
Name: 고객번호, Length: 18866, dtype: object